In [1]:
# run the following from the terminal
#sudo apt-get install sox libsndfile1 ffmpeg

In [2]:
!pip install wget unidecode
!pip3.8 install pytorch_lightning

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip3.8 install --upgrade pip
!pip3.8 install torch
BRANCH = 'r1.0.0rc1'
!python3.8 -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[tts]

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/NVIDIA/NeMo.git (to revision r1.0.0rc1) to /tmp/pip-install-2uc9vwai/nemo-toolkit_6f1b61c1834f467aabb86c7f87e07660
  Running command git clone --filter=blob:none -q https://github.com/NVIDIA/NeMo.git /tmp/pip-install-2uc9vwai/nemo-toolkit_6f1b61c1834f467aabb86c7f87e07660
  Running command git checkout -b r1.0.0rc1 --track origin/r1.0.0rc1
  Switched to a new branch 'r1.0.0rc1'
  Branch 'r1.0.0rc1' set up to track remote branch 'r1.0.0rc1' from 'origin'.
  Resolved https://github.com/NVIDIA/NeMo.git to commit 977b9a2c8bee5f0575935cde9527b9fe2b7badfe
  Preparing metadata (setup.py) ... done


In [4]:
import os
import pandas as pd
import re
import librosa
from datasets import Dataset

In [5]:
data_directory = "../Badaga_Corpus-v.0.1.0/"
tagged_file = "Badaga-v0.1.0.xlsx"

In [6]:
tagged_file_path = os.path.join(data_directory, tagged_file)
data_frame = pd.read_excel(tagged_file_path)
data_frame.dropna(inplace=True)

gender = "F" # "M", "F"


data_frame["audio_file_name"] = data_frame["audio_file_name"].apply(lambda x: os.path.join(data_directory, "clips", x))

data_frame = data_frame[data_frame["gender"]==gender]

train_df = data_frame[data_frame["split_label"]!="test"]
test_df = data_frame[data_frame["split_label"]=="test"]

In [7]:
train_df

,items,translated_transcript,audio_file_name,translterated_script,user_id,gender,locale,split_label,duration
0,1.0,what is the recipe in home,../Badaga_Corpus-v.0.1.0/clips/F002_1_1.mp3,manaya aena udhaka,F002,F,ba,train,2.377187
1,2.0,who is there near you,../Badaga_Corpus-v.0.1.0/clips/F002_1_2.mp3,pakka dhara edhdharae,F002,F,ba,train,2.377187
2,3.0,what did you prepare for lunch,../Badaga_Corpus-v.0.1.0/clips/F002_1_3.mp3,hagulu hasuga aena maditha,F002,F,ba,train,2.351062
3,4.0,did you brush,../Badaga_Corpus-v.0.1.0/clips/F002_1_4.mp3,hallu ujjithaya,F002,F,ba,train,2.194313
4,5.0,did you eat,../Badaga_Corpus-v.0.1.0/clips/F002_1_5.mp3,nee thindhubutaya,F002,F,ba,train,2.272687
...,...,...,...,...,...,...,...,...,...
9830,593.0,i dont cheat anybody,../Badaga_Corpus-v.0.1.0/clips/F004_593.mp3,na tharuvavu aemaathathu ellae,F004,F,ba,train,2.011437
9831,594.0,today it is so windy,../Badaga_Corpus-v.0.1.0/clips/F004_594.mp3,gaai jaasthi endhdhu,F004,F,ba,train,1.358375
9834,597.0,today is my birthday,../Badaga_Corpus-v.0.1.0/clips/F004_597.mp3,endhdhu aenna utti jaenna,F004,F,ba,train,1.541250
9835,598.0,we are coming tomorrow.,../Badaga_Corpus-v.0.1.0/clips/F004_598.mp3,enga naaiga bannaeyo,F004,F,ba,train,1.724125


In [8]:
import json

audio_paths = list(train_df["audio_file_name"])
durations = list(train_df["duration"])
labels = list(train_df["translterated_script"])


my_dict = dict()
i = 0
with open("files/tts_train.json", "w") as outfile:
    for path, dur, label in zip(audio_paths, durations, labels):
        my_dict = {"audio_filepath": path,
                "duration": dur,
                "text": label}
        json_object = json.dumps(my_dict)
        outfile.write(json_object + "\n")
        


audio_paths = list(test_df["audio_file_name"])
durations = list(test_df["duration"])
labels = list(test_df["translterated_script"])


my_dict = dict()
i = 0
with open("files/tts_test.json", "w") as outfile:
    for path, dur, label in zip(audio_paths, durations, labels):
        my_dict = {"audio_filepath": path,
                "duration": dur,
                "text": label}
        json_object = json.dumps(my_dict)
        outfile.write(json_object + "\n")

In [ ]:
directory = "tactron2-rbg-badaga-tts-"+gender.lower()

if os.path.exists(directory):
    pass
else:
    os.makedirs(directory)

!python3.8 tacotron2.py sample_rate=16000 train_dataset=files/tts_train.json validation_datasets=files/tts_test.json exp_manager.exp_dir=$directory trainer.max_epochs=20 trainer.accelerator=null trainer.check_val_every_n_epoch=1


[NeMo W 2021-12-18 15:16:08 experimental:27] Module <class 'nemo.collections.asr.data.audio_to_text_dali.AudioToCharDALIDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

[NeMo W 2021-12-18 15:16:08 nemo_logging:349] /home/ubuntu/.local/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/


Validating:   7%|██                              | 6/92 [00:16<02:53,  2.02s/it][NeMo W 2021-12-18 15:24:30 tacotron2:366] Reached max decoder steps 1000.

Epoch 0:  86%|▊| 533/617 [08:05<01:16,  1.10it/s, loss=2, v_num=6-09, training_l[NeMo W 2021-12-18 15:24:31 tacotron2:366] Reached max decoder steps 1000.

Validating:   9%|██▊                             | 8/92 [00:19<02:23,  1.71s/it][NeMo W 2021-12-18 15:24:33 tacotron2:366] Reached max decoder steps 1000.

Epoch 0:  87%|▊| 535/617 [08:08<01:14,  1.10it/s, loss=2, v_num=6-09, training_l[NeMo W 2021-12-18 15:24:34 tacotron2:366] Reached max decoder steps 1000.

Validating:  11%|███▎                           | 10/92 [00:22<02:09,  1.57s/it][NeMo W 2021-12-18 15:24:36 tacotron2:366] Reached max decoder steps 1000.

Epoch 0:  87%|▊| 537/617 [08:11<01:13,  1.09it/s, loss=2, v_num=6-09, training_l[NeMo W 2021-12-18 15:24:37 tacotron2:366] Reached max decoder steps 1000.

Validating:  13%|████                           | 12/92 [00:25<

Epoch 0:  94%|▉| 583/617 [09:17<00:32,  1.05it/s, loss=2, v_num=6-09, training_l[NeMo W 2021-12-18 15:25:44 tacotron2:366] Reached max decoder steps 1000.

Validating:  63%|███████████████████▌           | 58/92 [01:31<00:49,  1.45s/it][NeMo W 2021-12-18 15:25:45 tacotron2:366] Reached max decoder steps 1000.

Epoch 0:  95%|▉| 585/617 [09:20<00:30,  1.04it/s, loss=2, v_num=6-09, training_l[NeMo W 2021-12-18 15:25:47 tacotron2:366] Reached max decoder steps 1000.

Validating:  65%|████████████████████▏          | 60/92 [01:34<00:46,  1.45s/it][NeMo W 2021-12-18 15:25:48 tacotron2:366] Reached max decoder steps 1000.

Epoch 0:  95%|▉| 587/617 [09:23<00:28,  1.04it/s, loss=2, v_num=6-09, training_l[NeMo W 2021-12-18 15:25:49 tacotron2:366] Reached max decoder steps 1000.

Validating:  67%|████████████████████▉          | 62/92 [01:37<00:43,  1.45s/it][NeMo W 2021-12-18 15:25:51 tacotron2:366] Reached max decoder steps 1000.

Epoch 0:  95%|▉| 589/617 [09:26<00:26,  1.04it/s, loss=2, v_num=